In [1]:

import torch
import os
from torch.utils.data import RandomSampler
import torch.nn.functional as F
import time

In [2]:
import import_ipynb
import data_processing
import custom_model

<string>:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead


Image batch shape: torch.Size([16, 3, 128, 128])
Label batch shape: torch.Size([16])
First batch of images: tensor([[[[0.3225, 0.3225, 0.3225,  ..., 0.3225, 0.3194, 0.3194],
          [0.3255, 0.3255, 0.3255,  ..., 0.3225, 0.3194, 0.3194],
          [0.3255, 0.3255, 0.3255,  ..., 0.3225, 0.3194, 0.3194],
          ...,
          [0.2562, 0.2592, 0.2562,  ..., 0.5032, 0.4972, 0.4912],
          [0.2411, 0.2562, 0.2562,  ..., 0.5123, 0.4972, 0.4972],
          [0.2471, 0.2562, 0.2562,  ..., 0.5093, 0.4972, 0.5032]],

         [[0.3164, 0.3164, 0.3164,  ..., 0.3164, 0.3134, 0.3134],
          [0.3194, 0.3194, 0.3194,  ..., 0.3164, 0.3134, 0.3134],
          [0.3194, 0.3194, 0.3194,  ..., 0.3164, 0.3134, 0.3134],
          ...,
          [0.2532, 0.2562, 0.2532,  ..., 0.5002, 0.4972, 0.4972],
          [0.2381, 0.2532, 0.2532,  ..., 0.5093, 0.4942, 0.4972],
          [0.2441, 0.2532, 0.2532,  ..., 0.5062, 0.4942, 0.5032]],

         [[0.3496, 0.3496, 0.3496,  ..., 0.3496, 0.3466, 0.3466],


In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
NUM_EPOCHS = 3

model = custom_model.resnet18(num_classes=10)

model = model.to(DEVICE)

#原先这里选用SGD训练，但是效果很差，换成Adam优化就好了
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:
valid_loader = data_processing.test_loader
test_loader = data_processing.test_loader
train_loader = data_processing.train_loader


In [6]:
for images, labels in test_loader:
    print("Labels:", labels)
    print("Unique values in labels:", labels.unique())
    break  # 只查看一个批次的数据

Labels: tensor([0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1])
Unique values in labels: tensor([0, 1])


In [7]:
def compute_accuracy_and_loss(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    cross_entropy = 0.
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        cross_entropy += F.cross_entropy(logits, targets).item()
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100, cross_entropy/num_examples
    

start_time = time.time()
train_acc_lst, valid_acc_lst = [], []
train_loss_lst, valid_loss_lst = [], []

for epoch in range(NUM_EPOCHS):
    
    model.train()
    
    for batch_idx, (features, targets) in enumerate(train_loader):
    
        ### PREPARE MINIBATCH
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 500:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:04d}/{len(train_loader):04d} |' 
                   f' Cost: {cost:.4f}')

    # no need to build the computation graph for backprop when computing accuracy
    model.eval()
    with torch.set_grad_enabled(False):
        train_acc, train_loss = compute_accuracy_and_loss(model, train_loader, device=DEVICE)
        valid_acc, valid_loss = compute_accuracy_and_loss(model, valid_loader, device=DEVICE)
        train_acc_lst.append(train_acc)
        valid_acc_lst.append(valid_acc)
        train_loss_lst.append(train_loss)
        valid_loss_lst.append(valid_loss)
        print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} Train Acc.: {train_acc:.2f}%'
              f' | Validation Acc.: {valid_acc:.2f}%')
        
    elapsed = (time.time() - start_time)/60
    print(f'Time elapsed: {elapsed:.2f} min')
  
elapsed = (time.time() - start_time)/60
print(f'Total Training Time: {elapsed:.2f} min')

Epoch: 001/003 | Batch 0000/1248 | Cost: 2.3691
Epoch: 001/003 | Batch 0500/1248 | Cost: 0.7010
Epoch: 001/003 | Batch 1000/1248 | Cost: 0.2624
Epoch: 001/003 Train Acc.: 86.33% | Validation Acc.: 85.70%
Time elapsed: 15.65 min
Epoch: 002/003 | Batch 0000/1248 | Cost: 0.8019
Epoch: 002/003 | Batch 0500/1248 | Cost: 0.5110
Epoch: 002/003 | Batch 1000/1248 | Cost: 0.3778


KeyboardInterrupt: 